In [1]:

# pip install langdetect
# # pip install sentencepiece
# pip install awscli
# pip install boto3
# pip install --upgrade gradio

In [2]:
import gradio as gr
from transformers import pipeline, AutoTokenizer, TFAutoModelForSeq2SeqLM, MarianMTModel, MarianTokenizer
from dotenv import load_dotenv
import os
import subprocess
import torch
#Google Text to Speech
from gtts import gTTS
import tempfile
from langdetect import detect

import boto3

In [3]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# This function transcribes audio to text using Whisper in the original language it was spoken
def transcribe_audio_original(audio_filepath):
    try:
        transcription_pipeline = pipeline("automatic-speech-recognition", model="openai/whisper-large")
        transcription_result = transcription_pipeline(audio_filepath)
        transcribed_text = transcription_result['text']
        return transcribed_text
    except Exception as e:
        print(f"an error occured: {e}")
        return "Error in transcription"

In [5]:

# This function transcribes audio to text and then translates it into the specified language
def translate(transcribed_text, target_lang="es"):
    try:
        #Define the model and tokenizer
        src_lang = detect(transcribed_text)
        model_name =f"Helsinki-NLP/opus-mt-{src_lang}-{target_lang}"
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        
        #tokenize the text
        encoded_text = tokenizer(transcribed_text, return_tensors="pt", padding=True)
        
        #generate translation using the model
        translated_tokens = model.generate(**encoded_text)
        
        #decode the translated tokens
        translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        
        return translated_text
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Error in transcription or translation"

In [6]:
# Define function to translate text to speech for output
# Uses Google Text-to-speech

def text_to_speech(text):
    tts = gTTS(text, lang='en')
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
    tts.save(temp_file.name)
    return temp_file.name

In [7]:
# Create a voice map so that correct voice is selected based on target language.

voice_map = {
    "ar": "Hala",
    "en": "Gregory",
    "es": "Mia",
    "fr": "Liam",
    "de": "Vicki",
    "it": "Bianca",
    "zh": "Hiujin",
    "hi": "Kajal",
    "jap": "Tomoko",
    "trk": "Burcu"
    
}

In [8]:
# Define text-to-speech function using Amazon Polly

def polly_text_to_speech(text, lang_code):
    
    try:
    
        #get the appropriate voice ID from the mapping
        voice_id = voice_map[lang_code]
        
        #initialize boto3 client for polly
        polly_client = boto3.client('polly')
        
        #request speech synthesis
        response = polly_client.synthesize_speech(
            Engine = 'neural',
            Text=text,
            OutputFormat='mp3',
            VoiceId=voice_id
        )
        
        # Save the audio to a temporary file and return its path
        if "AudioStream" in response:
            with tempfile.NamedTemporaryFile(delete=False, suffix='.mp3') as audio_file:
                audio_file.write(response['AudioStream'].read())
                return audio_file.name
    except boto3.exceptions.Boto3Error as e:
        print(f"Error accessing Polly: {e}")
    return None  # Return None if there was an error
    

In [9]:
# Create a language map from full names to ISO codes
language_map = {
    "Arabic (Gulf)": "ar",
    "Chinese (Cantonese)": "zh",
    "English": "en",
    "French": "fr",
    "German": "de",
    "Hindi": "hi",
    "Italian": "it",
    "Japanese": "jap",
    "Spanish": "es",
    "Turkish": "trk"
    
}

In [10]:
def combined_function (audio_filepath, target_lang):
    
    #language detection for the original text
    transcribed_text = transcribe_audio_original(audio_filepath)
    detected_lang = detect(transcribed_text)
    
    #text to speech for original language
    speech = polly_text_to_speech(transcribed_text, detected_lang)
    
    #translation and text to speech for target language
    target_lang = language_map[target_lang]
    translation = translate(transcribed_text, target_lang)
    translated_speech = polly_text_to_speech(translation, target_lang)
    
    return transcribed_text, translation, speech, translated_speech

In [35]:
default_language = "English"
# default_index = list(language_map.keys()).index(default_language)

def transcribe_and_speech(audio_filepath=None, typed_text=None, target_lang=default_language):
    
    #Determine source of text: audio transctiption or direct text input
    if audio_filepath and typed_text:
        return "Please use only one input method at a time", None, None, None
    elif typed_text:
        query_text = typed_text
    elif audio_filepath:
        query_text = transcribe_audio_original(audio_filepath)
    else:
        return "Please provide input by typing or speaking.", None, None, None
    
    #Detect language of input text
    detected_lang_code = detect(query_text)
    detected_lang = [key for key, value in language_map.items() if value == detected_lang_code][0]
    
    #Check if the language is specified. Default to English if not.
    target_lang_code = language_map.get(target_lang, "en")
    
    #Process text: translate 
    #Check if the detected language and target language are the same
    # if detected_lang == target_lang:
    #     translated_text = query_text
    # else:
    #     translated_text = translate(query_text, target_lang_code)
    
    #convert to speech
    original_speech = polly_text_to_speech(query_text, detected_lang_code)
    # translated_speech = polly_text_to_speech(translated_text, target_lang_code)
    
    return query_text, original_speech

In [42]:
default_language = "English"
# default_index = list(language_map.keys()).index(default_language)



def translate_and_speech(audio_filepath=None, typed_text=None, target_lang=default_language):
    
    #Determine source of text: audio transctiption or direct text input
    if audio_filepath and typed_text:
        return "Please use only one input method at a time", None, None, None
    elif typed_text:
        query_text = typed_text
    elif audio_filepath:
        query_text = transcribe_audio_original(audio_filepath)
    else:
        return "Please provide input by typing or speaking.", None, None, None
    
    #Detect language of input text
    detected_lang_code = detect(query_text)
    detected_lang = [key for key, value in language_map.items() if value == detected_lang_code][0]
    
    #Check if the language is specified. Default to English if not.
    target_lang_code = language_map.get(target_lang, "en")
    
    #Process text: translate 
    #Check if the detected language and target language are the same
    if detected_lang == target_lang:
        translated_text = query_text
    else:
        translated_text = translate(query_text, target_lang_code)
    
    #convert to speech
    original_speech = polly_text_to_speech(query_text, detected_lang_code)
    translated_speech = polly_text_to_speech(translated_text, target_lang_code)
    
    return  translated_text, translated_speech

In [37]:
# Function to clear out all inputs

def clear_inputs():
    return None, None, None, None, None, None

In [45]:
# Create Gradio user interface for combined_function_2 allowing typed input or spoken input.

with gr.Blocks() as app2:
    
    with gr.Row():
        gr.Markdown("### Audio Transcription and Text to Speech")
        gr.Markdown("Record your question and choose a translation question")
    
    with gr.Row():
        input_audio = gr.Audio(
            label="Record your audio",
            type="filepath")
        input_text = gr.Textbox(
            label="Or type your question", 
            placeholder="Type here...")
        language_dropdown = gr.Dropdown(label="Click the middle of the dropdown bar to select translation language",
                                        choices=list(language_map.keys()), value=default_language, type='value')
    
    transcribe_button = gr.Button("Transcribe")
    translate_button = gr.Button("Translate")
    clear_button = gr.Button("Clear")
    
    output_transcribed = gr.Textbox(label="Transcribed text")
    output_original_speech = gr.Audio(label="Audio of original text")
    output_translated = gr.Textbox(label="Translated text")
    output_translated_speech = gr.Audio(label="Audio of translated text")
    
    # Audio transcription
    transcribe_button.click(
        fn=transcribe_and_speech,
        inputs=[input_audio, input_text, language_dropdown],
        outputs=[output_transcribed, output_original_speech]
    )
    
    # Translation
    translate_button.click(
        fn=translate_and_speech,
        inputs=[input_audio, input_text, language_dropdown],
        outputs=[output_translated, output_translated_speech]
    )
    
    #Clearing all inputs and outputs
    clear_button.click(
    fn=clear_inputs,
    inputs=[],
    outputs=[input_audio, input_text, output_transcribed, output_original_speech, output_translated, output_translated_speech]
    )
        
    # with gr.Row():
    #     submit_button = gr.Button("Submit")
        
    
        
    # output_query = gr.Textbox(label="Original text")
    # output_translated = gr.Textbox(label="Translated text")
    
    
    # submit_button.click(
    #     fn=combined_function_2,
    #     inputs=[input_audio, input_text, language_dropdown],
    #     outputs=[output_query, output_translated, output_original_speech, output_translated_speech]
    # )

app2.launch(show_error=True, share=True)

Running on local URL:  http://127.0.0.1:7873
Running on public URL: https://89b5d4904fb7c0512f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\mered\anaconda3\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 411, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mered\anaconda3\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mered\anaconda3\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "c:\Users\mered\anaconda3\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "c:\Users\mered\anaconda3\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "c:\Users\mered\anaconda3\Lib\site-packages\starlett

In [ ]:
# help(gr.Dropdown)

In [ ]:
# list of languages and their codes for dropdown
languages = gr.Dropdown(label="Click in the middle of the dropdown bar to select translation language", choices=list(language_map.keys()))

In [ ]:
# Create Gradio app to:
# 1. transcribe spoken audio to text
# 2. output transcribed text as speech

input_audio = gr.Audio(
            label="click on microphone to record audio in any language", 
            type="filepath", 
            #vaveform options customize the color of the wave seen when recording/playing.
            waveform_options = gr.WaveformOptions(
                waveform_color="#01C6FF",
                waveform_progress_color="#0066B4",
                skip_length=2,
                show_controls=False,
            ),
)

app = gr.Interface(
    fn=combined_function,
    inputs=[input_audio, languages],
    outputs=[
        gr.Textbox(label="Transcribed audio in original language"),
        gr.Textbox(label="Translated text in selected language"),
        gr.Audio(label="Audio speech in original language"),
        gr.Audio(label="Audio speech translated to selected language")],
    title="Audio Transcription and Text to Speech",
    description="Record your question and choose a translation language.")
    
app.launch(show_error=True, share=True) #uncomment share=True in google colab